# Workshop: De cero a Agente con LangChain y Python

En este workshop de 4 horas aprenderás a construir un agente inteligente desde cero utilizando **LangChain**, **Python** y modelos de lenguaje **open source** como **Ollama** o **LM Studio**. A lo largo del taller veremos los conceptos básicos de los LLMs, cómo orquestar herramientas y memorias con LangChain, y cómo integrar un vector store para recuperación aumentada de datos (RAG).

## Agenda del Workshop

1. Introducción a los LLMs y la ejecución local
2. Componentes de LangChain (modelos, chains, herramientas)
3. Definición de herramientas personalizadas
4. Configuración de un modelo local (Ollama / LM Studio)
5. Creación de un agente reactivo con memoria
6. Integración con un vector store para RAG
7. Demostración final y conclusiones


## Conexión directa a un LLM sin LangChain

Antes de introducir LangChain, veamos cómo podríamos interactuar con un modelo de lenguaje utilizando únicamente el SDK o la API que proporciona el modelo. Por ejemplo, un servidor de Ollama expone un endpoint HTTP `http://localhost:11434/api/generate` donde puedes enviar un prompt y recibir la respuesta del modelo en formato JSON. De forma análoga, otros proveedores (como OpenAI) ofrecen SDKs o endpoints REST para invocar sus modelos.

Interactuar de forma directa es útil para pruebas sencillas, pero pronto verás que gestionar memoria de conversación, combinar varios modelos, reintentar peticiones o integrar fuentes de datos externas se vuelve complejo. Aquí es donde entra LangChain.

In [ ]:
# Ejemplo de llamada directa a un modelo local de Ollama
import requests

# Definimos el payload de la solicitud
data = {
    "model": "llama3:8b",
    "prompt": "Hola, ¿cómo estás?",
    "stream": False
}

# Realizamos la petición POST al endpoint de Ollama
# (Nota: esta llamada sólo funcionará si tienes ollama corriendo de forma local)
# response = requests.post("http://localhost:11434/api/generate", json=data)
# print(response.json())

# Con otros proveedores usarías sus SDK o un endpoint similar. Aquí simplemente mostramos el cuerpo de la solicitud.

# Ejemplo de llamada a un modelo de OpenAI
import openai

# Definimos el payload de la solicitud
data = {
    "model": "gpt-4o",
    "prompt": "Hola, ¿cómo estás?",
    "stream": False
}

# Realizamos la petición POST al endpoint de OpenAI
response = openai.ChatCompletion.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "Hola, ¿cómo estás?"}]
)
print(response.choices[0].message.content)

# ejemplo de llamado a google gemini
import google.generativeai as genai

# Definimos el payload de la solicitud
data = {
    "model": "gemini-2.0-flash",
    "prompt": "Hola, ¿cómo estás?",
    "stream": False
}

# Realizamos la petición POST al endpoint de Google Gemini
response = genai.generate_content(data)
print(response.text)

# ejemplo de llamado a openai


## ¿Por qué utilizar LangChain?

Aunque podrías interactuar directamente con un modelo de lenguaje usando el SDK que ofrece cada proveedor (por ejemplo la API de **OpenAI**, el servidor local de **Ollama** o el endpoint de **LM Studio**), **LangChain** proporciona una capa de abstracción y orquestación muy útil cuando necesitas construir agentes más sofisticados:

- **Unifica interfaces**: te permite cambiar entre distintos LLMs (open source o propietarios) sin modificar el resto de tu código, porque expone una API común para modelos de chat, embeddings y vector stores.
- **Encadena tareas**: facilita construir *chains* donde la salida de una llamada se usa como entrada de otra, incluyendo flujos de preguntas y respuestas, análisis de datos o ejecución de herramientas externas.
- **Gestión de memoria**: ofrece componentes para almacenar el historial de conversaciones y recuperarlo, algo esencial para agentes conversacionales.
- **Integración de herramientas**: permite exponer funciones personalizadas (cálculos, búsquedas, consultas API, etc.) como herramientas que el modelo puede invocar cuando es necesario.
- **Recuperación aumentada (RAG)**: se integra con motores de vector y sistemas de embeddings para buscar documentos relevantes y combinarlos con la generación del LLM.

En resumen, LangChain actúa como el pegamento que conecta los diferentes bloques (modelos, herramientas, bases de datos) y te permite centrarte en la lógica de tu agente en lugar de los detalles de cada SDK.
    

### Instalación y configuración

Para seguir este notebook necesitas instalar varias librerías. Si ya tienes un entorno con `langchain` y `chromadb` puedes omitir esta celda. En una máquina local con acceso a internet se pueden instalar así:

In [ ]:
!pip install --upgrade pip
!pip install langchain langchain-community langchain-core chromadb sentence-transformers

### Importación de módulos

Importamos las clases y funciones necesarias para construir el agente. Esto incluye el modelo local (por ejemplo `ChatOllama`), el motor de memoria, las herramientas y funciones auxiliares de LangChain.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool, tool
from langchain_community.chat_models import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.agents import create_react_agent, AgentExecutor

## Cadenas en LangChain

Una **cadena** combina uno o más componentes (prompts, modelos, transformaciones) para construir un flujo de ejecución. LangChain incluye utilidades como `LLMChain` para encapsular un prompt y un modelo. Aquí tienes un ejemplo de cadena simple que genera una respuesta a partir de un template:

In [ ]:
# Ejemplo de cadena simple
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain.chains import LLMChain

# Definimos un prompt
prompt = ChatPromptTemplate.from_template("Dime un dato curioso sobre {tema}.")

# Instanciamos el modelo local (suponiendo que esté en marcha)
llm_local = ChatOllama(model="llama3:8b", base_url="http://localhost:11434")

# Creamos la cadena
cadena = llm_local | prompt

# Para ejecutarla proporcionaríamos las variables del template:
resultado = cadena.invoke({"tema": "Colombia"})
# print(resultado)
# Nota: descomenta estas líneas para ejecutar con un modelo local en marcha.

## Memoria en LangChain

Los agentes conversacionales necesitan recordar lo que ya se ha dicho. LangChain ofrece varias implementaciones de memoria, como `ConversationBufferMemory`, que almacena el historial de mensajes en orden. Puedes combinarla con un LLMChain o un agente para que el modelo reciba contexto en cada llamada.

In [ ]:
# Uso de ConversationBufferMemory con un LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

# Creamos la memoria
memoria_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Definimos el prompt
prompt_memoria = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente amistoso."),
    ("human", "{input}"),
    ("ai", "{chat_history}"),
])

# Creamos la cadena con memoria
cadena_memoria = LLMChain(llm=llm_local, prompt=prompt_memoria, memory=memoria_chain)

# Para usarla, invoca la cadena varias veces; la memoria conservará el historial:
# respuesta1 = cadena_memoria.invoke({"input": "Hola"})
# respuesta2 = cadena_memoria.invoke({"input": "¿Qué me dijiste antes?"})
# print(respuesta2)


## Plantillas de Prompt (Prompt Templates)

LangChain facilita la construcción de prompts complejos mediante plantillas parametrizadas. Puedes combinar mensajes de sistema, de usuario y del asistente para definir el comportamiento del modelo.

In [ ]:
# Ejemplo de plantilla de prompt
from langchain_core.prompts import ChatPromptTemplate

plantilla = ChatPromptTemplate.from_messages([
    ("system", "Eres un experto en matemáticas."),
    ("human", "Pregunta: {pregunta}"),
])

# Instanciaríamos el modelo y llamaríamos:
# llm_local = ChatOllama(model="llama3:8b", base_url="http://localhost:11434")
# chain = LLMChain(llm=llm_local, prompt=plantilla)
# respuesta = chain.invoke({"pregunta": "¿Cuánto es 12×8?"})
# print(respuesta)


## Utilizar diferentes modelos con la misma lógica

Gracias a las capas de abstracción de LangChain, puedes cambiar de un modelo local (por ejemplo, vía `ChatOllama`) a un modelo alojado (por ejemplo, `ChatOpenAI`) sin reescribir el flujo de tu aplicación. Sólo debes cambiar la clase del modelo cuando lo instancias.

In [ ]:
# Ejemplo de inicialización con un modelo de OpenAI
from langchain_openai import ChatOpenAI

# Suponiendo que tengas configurada la API Key en tu entorno
# openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# La cadena o el agente podrían usar indistintamente openai_llm en lugar de llm_local


## 1. Definir herramientas personalizadas

Las **herramientas** permiten que el agente ejecute funciones específicas (por ejemplo cálculos o búsquedas). Definiremos dos funciones sencillas que suman y multiplican números. Utilizamos el decorador `@tool` para convertirlas en herramientas que LangChain pueda invocar.

In [ ]:
from langchain_core.tools import tool

@tool
def sumar(a: float, b: float) -> float:
    # Suma dos números y devuelve el resultado
    return a + b

@tool
def multiplicar(a: float, b: float) -> float:
    # Multiplica dos números y devuelve el resultado
    return a * b

# Registrar las herramientas en una lista
herramientas = [sumar, multiplicar]

## 2. Instanciar el modelo local

Antes de crear el agente necesitamos un LLM local en ejecución. Con **Ollama** podemos iniciar un servidor y cargar un modelo como *llama3:8b*:

```bash
# Instala Ollama (solo una vez)
wget -qO- https://ollama.com/install.sh | sh
# Arranca el servidor
ollama serve &
# Descarga y prepara el modelo (puede tardar unos minutos)
ollama pull llama3:8b
```

En **LM Studio** puedes descargar modelos desde la interfaz gráfica y exponer un endpoint local. Una vez en marcha, LangChain se conecta mediante la clase `ChatOllama` indicando el nombre del modelo.

In [ ]:
# Configuramos el modelo local (asegúrate de que el servidor de Ollama esté ejecutándose)
# Si utilizas LM Studio, cambia el nombre del modelo o ajusta el endpoint.
llm = ChatOllama(model="llama3:8b", base_url="http://localhost:11434")

# También puedes ajustar parámetros como temperatura, top_p, etc.
# llm = ChatOllama(model="llama3:8b", temperature=0.5)

## 3. Crear memoria y construir el agente

La memoria mantiene el historial de conversación. Usaremos `ConversationBufferMemory` para recordar los mensajes pasados. Luego construiremos un **agente reactivo** con `create_react_agent`, pasando el modelo, la lista de herramientas y la memoria.

In [ ]:
# Crear una memoria de conversación
memoria = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Crear el prompt que usará el agente
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente útil que puede usar herramientas para completar tareas."),
    ("human", "{input}"),
    ("agent", "{agent_scratchpad}"),
])

# Crear el agente reactivo (ReAct)
agente = create_react_agent(llm, herramientas, prompt)

# Ejecutar el agente dentro de un executor para gestionar el estado y la memoria
executor = AgentExecutor(agent=agente, tools=herramientas, memory=memoria, verbose=True)

## 4. Ejecutar el agente

Ya podemos hacer consultas al agente. El agente decidirá si necesita llamar a alguna herramienta para calcular o buscar información.

In [ ]:
# Ejemplo de consulta: combinación de cálculo
pregunta = {"messages": [{"role": "user", "content": "¿Cuánto es 5*7 y 3*8?"}]}
# Para probar el agente descomenta las líneas siguientes cuando tengas el modelo local ejecutándose:
# respuesta = executor.invoke(pregunta)
# print(respuesta)

## 5. Integrar un vector store para Recuperación Aumentada (RAG)

Para responder preguntas sobre tus propios documentos, podemos crear una herramienta de recuperación de texto basada en embeddings. Utilizaremos **Chroma** como vector store y **HuggingFaceEmbeddings** como modelo de embedding.

Primero cargamos algunos textos de ejemplo y construimos el vector store. Luego exponemos una herramienta `search_docs` que busca los documentos más relevantes y devuelve su contenido.

In [ ]:
# from langchain_community.vectorstores import Chroma
# from langchain_community.embeddings import HuggingFaceEmbeddings
#
# textos = [
#     "Colombia es un país situado en América del Sur cuya capital es Bogotá.",
#     "La multiplicación es una operación matemática que suma un número consigo mismo muchas veces.",
# ]
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# store = Chroma.from_texts(textos, embeddings)
# retriever = store.as_retriever()
#
# @tool
# def search_docs(query: str) -> str:
#     # Busca documentos relevantes y devuelve el texto más parecido
#     docs = retriever.get_relevant_documents(query)
#     return docs[0].page_content
#
# # Añadimos esta herramienta a la lista de herramientas del agente
# herramientas.append(search_docs)

## 6. Demostración final

Después de añadir la herramienta de recuperación, el agente puede combinar cálculo y búsqueda de datos. Por ejemplo:

```python
pregunta = {"messages": [{"role": "user", "content": "¿Cuál es la capital de Colombia y cuánto es 9×7?"}]}
# respuesta = executor.invoke(pregunta)
# print(respuesta)
```

El agente consultará la herramienta de búsqueda para obtener la capital (Bogotá) y la herramienta de multiplicación para calcular 9×7, devolviendo una respuesta completa.

## 7. Conclusiones y próximos pasos

En este notebook has visto cómo:

- Instalar y configurar las dependencias necesarias para ejecutar modelos locales.
- Definir herramientas personalizadas con el decorador `@tool`.
- Inicializar un modelo local utilizando **Ollama** o **LM Studio**.
- Construir un agente reactivo con memoria y herramientas en LangChain.
- Integrar un vector store para realizar búsquedas en tus documentos (RAG).

**Próximos pasos:** explora otros modelos open source (Mistral, Mixtral), añade nuevas herramientas (APIs, bases de datos) y optimiza el flujo de tu agente con LangGraph.